In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [ ]:
!kaggle datasets download -d alfathterry/bbc-full-text-document-classification

Dataset URL: https://www.kaggle.com/datasets/alfathterry/bbc-full-text-document-classification
License(s): MIT
  0% 0.00/1.84M [00:00<?, ?B/s]
100% 1.84M/1.84M [00:00<00:00, 23.8MB/s]


In [ ]:
!mkdir data

In [ ]:
!unzip /content/bbc-full-text-document-classification.zip -d /content/data

Archive:  /content/bbc-full-text-document-classification.zip
  inflating: /content/data/bbc_data.csv  


In [ ]:
import numpy as np
import gensim.downloader
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
keras.utils.set_random_seed(42)

In [ ]:
df=pd.read_csv("/content/data/bbc_data.csv",encoding='unicode_escape', on_bad_lines='skip')

In [ ]:
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2225 non-null   object
 1   labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
np.unique(df["labels"].values)

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [ ]:
df.isnull().values.any()

False

In [ ]:
fig = px.histogram(df, x=df['data'].apply(lambda x: x.split()).str.len().values ,nbins=50)
fig.update_layout(
    title="Sentences",
    xaxis_title="Sentences with number of words",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [ ]:
fig = px.histogram(df, x=df['labels'] ,nbins=50)
fig.update_layout(
    title="Sentences",
    xaxis_title="labels",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

**Data preprocessing**

In [ ]:
max_seq=450

In [ ]:
df=df[df['data'].apply(lambda x: x.split()).str.len().values<=max_seq]

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))
stopWords=stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
def remove(sentence,stopwords):
  newSentence=""
  for word in sentence.split():
    if(word not in stopwords):
      newSentence+=word+" "
  return newSentence

In [ ]:
#df['data']=df['data'].apply(lambda x: re.sub("[^0-9a-z\s]",'',x.lower())).apply(lambda sentence: " ".join([word for word in sentence.split() if word not in stopWords]))
df['data']=df['data'].apply(lambda x: re.sub("[^0-9a-z\s]",'',x.lower())).apply(lambda sentence: " ".join([word for word in sentence.split()]))
df['encode']=df['labels'].astype('category').cat.codes.astype('int32')

In [ ]:
df.head()

,data,labels,encode
0,musicians to tackle us red tape musicians grou...,entertainment,1
2,rocker doherty in onstage fight rock singer pe...,entertainment,1
3,snicket tops us box office chart the film adap...,entertainment,1
4,oceans twelve raids box office oceans twelve t...,entertainment,1
5,landmark movies of 2004 hailed us film profess...,entertainment,1


In [ ]:
len(df)

1609

In [ ]:
#!pip install transformers==4.37.*

In [ ]:
from transformers import BertTokenizerFast,TFBertModel

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
bert_model.config_class

transformers.models.bert.configuration_bert.BertConfig

In [ ]:
data_tokens=tokenizer.batch_encode_plus(df['data'].tolist(), padding=True,return_tensors = 'np')

Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
max_seq=data_tokens["input_ids"].shape[1]

In [ ]:
labels=df['encode'].values

In [ ]:
labels.shape

(1609,)

In [ ]:
np.arange(labels.shape[0])

array([   0,    1,    2, ..., 1606, 1607, 1608])

In [ ]:
xIdxTrain,xIdxTest,_,_=train_test_split(np.arange(labels.shape[0]), np.arange(labels.shape[0]), test_size=0.15, random_state=42)

In [ ]:
xIdxTrain,xIdxval,_,_=train_test_split(xIdxTrain,xIdxTrain, test_size=0.1, random_state=42)

In [ ]:
x_train_input=data_tokens['input_ids'][xIdxTrain]
x_train_attention=data_tokens['attention_mask'][xIdxTrain]
x_train_type=data_tokens['token_type_ids'][xIdxTrain]
y_train=labels[xIdxTrain]
x_val_input=data_tokens['input_ids'][xIdxval]
x_val_attention=data_tokens['attention_mask'][xIdxval]
x_val_type=data_tokens['token_type_ids'][xIdxval]
y_val=labels[xIdxval]
x_test_input=data_tokens['input_ids'][xIdxTest]
x_test_attention=data_tokens['attention_mask'][xIdxTest]
x_test_type=data_tokens['token_type_ids'][xIdxTest]
y_test=labels[xIdxTest]

In [ ]:
print("x_train_input.shape= ",x_train_input.shape)
print(x_train_attention.shape)
print(x_train_type.shape)
print(y_train.shape)
print(x_test_input.shape)
print(x_test_attention.shape)
print(x_test_type.shape)
print(y_test.shape)

x_train_input.shape=  (1230, 588)
(1230, 588)
(1230, 588)
(1230,)
(242, 588)
(242, 588)
(242, 588)
(242,)


In [ ]:
xx=tokenizer(df['data'].tolist()[0], padding=True,return_tensors = 'np')

In [ ]:
def bert_layer(inputs):
    inputT, inputA,inputS=inputs
    output=bert_model(input_ids=inputT,attention_mask = inputA,token_type_ids=inputS,return_dict =True)
    return output["last_hidden_state"]

In [ ]:
inputT=layers.Input(shape=(max_seq,),dtype = 'int32')
inputA=layers.Input(shape=(max_seq,),dtype = 'int32')
inputS=layers.Input(shape=(max_seq,),dtype = 'int32')
bert_output =layers.Lambda(bert_layer,output_shape=(max_seq, 768))([inputT, inputA,inputS])
print(bert_output)
embedding = layers.Dropout(0.3)(bert_output)
avg=layers.GlobalAveragePooling1D()(embedding)
print(avg)
dense=layers.Dense(256)(avg)
batch=layers.BatchNormalization()(dense)
act=layers.Activation('relu')(batch)
dense1=layers.Dense(5)(act)
batch1=layers.BatchNormalization()(dense1)
output=layers.Activation('softmax')(batch1)
bertAVG=keras.Model([inputT,inputA,inputS],output)
print(bertAVG.layers[3])
bertAVG.layers[3].trainable=False
print(bertAVG.layers[3].trainable)
bertAVG.summary()
bertAVG.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

<KerasTensor shape=(None, 588, 768), dtype=float32, sparse=False, name=keras_tensor_3>
<KerasTensor shape=(None, 768), dtype=float32, sparse=False, name=keras_tensor_5>
<Lambda name=lambda, built=True>
False


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 588)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 588, 768)       │              0 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0],   │
│                           │                        │                │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 588, 768)       │              0 │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 768)            │              0 │ dropout[0][0]          │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        196,864 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 256)            │          1,024 │ dense[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 256)            │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 5)              │          1,285 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 5)              │             20 │ dense_1[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 5)              │              0 │ batch_normalization_1… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 199,193 (778.10 KB)

 Trainable params: 198,671 (776.06 KB)

 Non-trainable params: 522 (2.04 KB)

In [ ]:
bertAVG.fit([x_train_input,x_train_attention,x_train_type], y_train, epochs=20, batch_size=128  ,
          validation_data=([x_val_input,x_val_attention,x_val_type],y_val ) ,  verbose = 1)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 129s 9s/step - accuracy: 0.6294 - loss: 1.1177 - val_accuracy: 0.8029 - val_loss: 1.0366
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - accuracy: 0.9661 - loss: 0.4602 - val_accuracy: 0.9051 - val_loss: 0.9345
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.9686 - loss: 0.4262 - val_accuracy: 0.9197 - val_loss: 0.9424
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.9793 - loss: 0.4009 - val_accuracy: 0.9197 - val_loss: 0.9643
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.9870 - loss: 0.3809 - val_accuracy: 0.9197 - val_loss: 0.9746
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.9872 - loss: 0.3638 - val_accuracy: 0.9270 - val_loss: 0.9820
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 5s/step - accuracy: 0.9896 - loss: 0.3493 - val_accuracy: 0.9270 - val_loss: 0.9921
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.9939 - loss: 0.3350 - val_accuracy: 0.9270 - val_loss

In [ ]:
bertAVG.save('bertAVGclassification.keras')

In [ ]:
y_predict=bertAVG.predict([x_test_input,x_test_attention,x_test_type])
y_predict=np.argmax(y_predict,axis=-1)
print(classification_report(y_test,y_predict))

8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
              precision    recall  f1-score   support

           0       0.97      0.94      0.95        77
           1       1.00      1.00      1.00        51
           2       0.87      1.00      0.93        34
           3       1.00      1.00      1.00        54
           4       1.00      0.92      0.96        26

    accuracy                           0.97       242
   macro avg       0.97      0.97      0.97       242
weighted avg       0.97      0.97      0.97       242



In [ ]:
inputT=layers.Input(shape=(max_seq,),dtype = 'int32')
inputA=layers.Input(shape=(max_seq,),dtype = 'int32')
inputS=layers.Input(shape=(max_seq,),dtype = 'int32')
bert_output =layers.Lambda(bert_layer,output_shape=(max_seq, 768))([inputT, inputA,inputS])
print(bert_output)
embedding = layers.Dropout(0.3)(bert_output)
maxx=layers.GlobalMaxPooling1D()(embedding)
print(maxx)
dense=layers.Dense(256)(maxx)
batch=layers.BatchNormalization()(dense)
act=layers.Activation('relu')(batch)
dense1=layers.Dense(5)(act)
batch1=layers.BatchNormalization()(dense1)
output=layers.Activation('softmax')(batch1)
bertMAX=keras.Model([inputT,inputA,inputS],output)
print(bertMAX.layers[3])
bertMAX.layers[3].trainable=False
print(bertMAX.layers[3].trainable)
bertMAX.summary()
bertMAX.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

<KerasTensor shape=(None, 588, 768), dtype=float32, sparse=False, name=keras_tensor_15>
<KerasTensor shape=(None, 768), dtype=float32, sparse=False, name=keras_tensor_17>
<Lambda name=lambda_1, built=True>
False


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1 (Lambda)         │ (None, 588, 768)       │              0 │ input_layer_3[0][0],   │
│                           │                        │                │ input_layer_4[0][0],   │
│                           │                        │                │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 588, 768)       │              0 │ lambda_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 768)            │              0 │ dropout_1[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        196,864 │ global_max_pooling1d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 256)            │          1,024 │ dense_2[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 256)            │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 5)              │          1,285 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 5)              │             20 │ dense_3[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 5)              │              0 │ batch_normalization_3… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 199,193 (778.10 KB)

 Trainable params: 198,671 (776.06 KB)

 Non-trainable params: 522 (2.04 KB)

In [ ]:
bertMAX.fit([x_train_input,x_train_attention,x_train_type], y_train, epochs=20, batch_size=128  ,
          validation_data=([x_val_input,x_val_attention,x_val_type],y_val ) ,  verbose = 1)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 95s 8s/step - accuracy: 0.5309 - loss: 1.3149 - val_accuracy: 0.2263 - val_loss: 4.7103
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - accuracy: 0.9391 - loss: 0.5463 - val_accuracy: 0.4672 - val_loss: 1.2570
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9691 - loss: 0.4749 - val_accuracy: 0.8102 - val_loss: 0.7410
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9787 - loss: 0.4412 - val_accuracy: 0.6204 - val_loss: 0.9872
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9883 - loss: 0.4174 - val_accuracy: 0.6861 - val_loss: 0.9934
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9861 - loss: 0.3973 - val_accuracy: 0.8686 - val_loss: 0.7821
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9918 - loss: 0.3785 - val_accuracy: 0.9416 - val_loss: 0.6934
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9920 - loss: 0.3610 - val_accuracy: 0.9197 - val_loss:

In [ ]:
bertMAX.save('bertMAXclassification.keras')

In [ ]:
y_predict=bertMAX.predict([x_test_input,x_test_attention,x_test_type])
y_predict=np.argmax(y_predict,axis=-1)
print(classification_report(y_test,y_predict))

8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
              precision    recall  f1-score   support

           0       0.93      0.90      0.91        77
           1       0.98      0.94      0.96        51
           2       0.86      0.91      0.89        34
           3       0.89      1.00      0.94        54
           4       1.00      0.85      0.92        26

    accuracy                           0.93       242
   macro avg       0.93      0.92      0.92       242
weighted avg       0.93      0.93      0.93       242



In [ ]:
inputT=layers.Input(shape=(max_seq,),dtype = 'int32')
inputA=layers.Input(shape=(max_seq,),dtype = 'int32')
inputS=layers.Input(shape=(max_seq,),dtype = 'int32')
bert_output =layers.Lambda(bert_layer,output_shape=(max_seq, 768))([inputT, inputA,inputS])
print(bert_output)
embedding = layers.Dropout(0.3)(bert_output)
cls=embedding[:, 0, :]
print(cls)
dense=layers.Dense(256)(cls)
batch=layers.BatchNormalization()(dense)
act=layers.Activation('relu')(batch)
dense1=layers.Dense(5)(act)
batch1=layers.BatchNormalization()(dense1)
output=layers.Activation('softmax')(batch1)
bertCLS=keras.Model([inputT,inputA,inputS],output)
print(bertCLS.layers[3])
bertCLS.layers[3].trainable=False
print(bertCLS.layers[3].trainable)
bertCLS.summary()
bertCLS.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

<KerasTensor shape=(None, 588, 768), dtype=float32, sparse=False, name=keras_tensor_27>
<KerasTensor shape=(None, 768), dtype=float32, sparse=False, name=keras_tensor_29>
<Lambda name=lambda_2, built=True>
False


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_7             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 588)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 588, 768)       │              0 │ input_layer_6[0][0],   │
│                           │                        │                │ input_layer_7[0][0],   │
│                           │                        │                │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 588, 768)       │              0 │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 768)            │              0 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 256)            │        196,864 │ get_item[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 256)            │          1,024 │ dense_4[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 256)            │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 5)              │          1,285 │ activation_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 5)              │             20 │ dense_5[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_5 (Activation) │ (None, 5)              │              0 │ batch_normalization_5… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 199,193 (778.10 KB)

 Trainable params: 198,671 (776.06 KB)

 Non-trainable params: 522 (2.04 KB)

In [ ]:
bertCLS.fit([x_train_input,x_train_attention,x_train_type], y_train, epochs=20, batch_size=128  ,
          validation_data=([x_val_input,x_val_attention,x_val_type],y_val ) ,  verbose = 1)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 95s 8s/step - accuracy: 0.5632 - loss: 1.2011 - val_accuracy: 0.7299 - val_loss: 0.7507
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 111s 5s/step - accuracy: 0.9302 - loss: 0.5498 - val_accuracy: 0.8321 - val_loss: 0.5194
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.9524 - loss: 0.4996 - val_accuracy: 0.8686 - val_loss: 0.4479
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9647 - loss: 0.4662 - val_accuracy: 0.8905 - val_loss: 0.4273
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9649 - loss: 0.4483 - val_accuracy: 0.9051 - val_loss: 0.4291
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9748 - loss: 0.4265 - val_accuracy: 0.9051 - val_loss: 0.4387
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9791 - loss: 0.4112 - val_accuracy: 0.9051 - val_loss: 0.4490
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9803 - loss: 0.3950 - val_accuracy: 0.9197 - val_loss

In [ ]:
bertCLS.save('bertCLSclassification.keras')

In [ ]:
y_predict=bertCLS.predict([x_test_input,x_test_attention,x_test_type])
y_predict=np.argmax(y_predict,axis=-1)
print(classification_report(y_test,y_predict))

8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        77
           1       1.00      1.00      1.00        51
           2       0.92      1.00      0.96        34
           3       1.00      1.00      1.00        54
           4       1.00      1.00      1.00        26

    accuracy                           0.99       242
   macro avg       0.98      0.99      0.99       242
weighted avg       0.99      0.99      0.99       242

